In [2]:
import json
import string
import random 
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer 
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout
nltk.download("punkt")
from nltk.corpus import wordnet
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Mounted at /content/drive


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [3]:
# initializing lemmatizer to get stem of words
lemmatizer = WordNetLemmatizer()

In [4]:
word = 'caring'
print(lemmatizer.lemmatize(word, pos='v'))

care


In [5]:
df = pd.read_json (r'/content/file.json')

In [6]:
words = []
doc_X = []
doc_y = []
classes = []

In [7]:
for intent in df["intents"]:
  for pattern in intent["patterns"]:
    token = nltk.word_tokenize(pattern)
    words.extend(token)
    doc_X.append(pattern)
    doc_y.append(intent["tag"])

    if intent["tag"] not in classes:
      classes.append(intent["tag"])

In [8]:
words = [lemmatizer.lemmatize((word)) for word in words if word not in string.punctuation]

In [9]:
words = sorted(set(words))
classes = sorted(set(classes))

In [10]:
type(words)

list

In [11]:
training = []
out_empty = [0] * len(classes)
for idx, doc in enumerate(doc_X):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word.lower() in text else bow.append(0)
        output_row = list(out_empty)
        output_row[classes.index(doc_y[idx])] = 1
        training.append([bow, output_row])

In [ ]:
training

In [13]:
random.shuffle(training)
training = np.array(training, dtype=object)

In [14]:
train_x = np.array(list(training[:,0]))
train_y = np.array(list(training[:,1]))

In [ ]:
classes

In [ ]:
# defining some parameters
input_shape = (len(train_x[0]),)
output_shape = len(train_y[0])
epochs = 200
# the deep learning model
model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(output_shape, activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
print(model.summary())
model.fit(x=train_x, y=train_y, epochs=200, verbose=1)

In [17]:
def pred_class(message,wordstring,classes):
  text = lemmatizer.lemmatize(message.lower())
  bow_list = []
  for word in words:
    bow_list.append(1) if word.lower() in text else bow_list.append(0)
  result = model.predict(np.array([bow_list]))[0]
  thresh = 0.2
  y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh]
  y_pred.sort(key=lambda x: x[1], reverse=True) 
  y_pred = y_pred[:1]
  lables = classes[y_pred[0][0]]
  for intent in df["intents"]:
    if intent["tag"] == lables:
      result = random.choice(intent["responses"])
      break
  return result


In [21]:
# running the chatbot
while True:
  message = input("")
  result = pred_class(message, words, classes)
  print(result)

In [20]:
classes

['about',
 'complaint',
 'createaccount',
 'goodbye',
 'greeting',
 'help',
 'name',
 'thanks']

In [22]:
import pickle
with open('chatbot.pickle','wb') as f:
  pickle.dump(model,f)

INFO:tensorflow:Assets written to: ram://8e804bd1-a613-4da4-8114-47a2ee1f0f14/assets


In [23]:
with open('words.txt', 'w') as fp:
    for item in words:
        # write each item on a new line
        if words.index(item) == len(words)-1:
          fp.write("%s" % item)
        else:
          fp.write("%s," % item)
        

In [24]:
model.save(f"saved_model")

INFO:tensorflow:Assets written to: saved_model/assets


In [ ]:
from google.colab import files
files.download('saved_model') 

In [27]:
model_version=1
model.save(f"{model_version}")

INFO:tensorflow:Assets written to: 1/assets


In [28]:
!tar -czvf model.tar.gz 1/

1/
1/assets/
1/variables/
1/variables/variables.index
1/variables/variables.data-00000-of-00001
1/saved_model.pb
1/keras_metadata.pb
